In [1]:
# For downloading historical market data:
import yfinance as yf

# For data manipulation:
import pandas as pd

# For working with dates and times:
from datetime import datetime, timedelta

# For timezone support:
import pytz

# <font color='pink'>Utility Functions</font>


Removing seconds from datetime strings in a Dataframe.

In [2]:
def removeSeconds(df):
    modified_dates=[]
    for date in df['Date']:

        format = '%Y-%m-%d %H:%M:%S%z'

        # Converting stirng to datetime object using the above format:
        date = datetime.strptime(date, format)

        # Removing seconds from the datetime object:
        modified_date = date.replace(second=0)
        
        modified_dates.append(modified_date)
    df['Date']=modified_dates
    return df

Converting datetime values in a DataFrame to a specific timezone.

In [3]:
def convertTimezone(timezone,df,name):

    # The target timezone:
    target_tz = pytz.timezone(timezone)

    # If we're using DataFrame index:
    if df.index is not None:
        # Converting the datetime to the target timezone:
        df.index = pd.to_datetime(df.index, utc=True).tz_convert(timezone)

    # If we're not using DataFrame index:
    else:
        # Converting the datetime to the target timezone:
        df[name] = pd.to_datetime(df[name],utc=True).dt.tz_convert(target_tz)
    return df

# NOTES:
    # 'utc=True' for ensuring that all datetime values are in UTC before any operations
    # ... to avoid issues related to timezone conversions

Retrieving stock prices for a specified date range using Yahoo Finance API 'yfinance'.

In [4]:
def getStockPrices(start_date,end_date,ticker):

    # Converting start and end date to datetime objects:
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    # Interval for stock price data retrieval (1 minute):
    interval = "1m"                     

    # Assigning start_date to current_date:
    current_date = start_date
    
    # List that will contain all retrieved stock price data:
    data_list = []  

    todays_date = datetime.now()

    while current_date < end_date:

        # Since we're downloading stock price data in chunks of up to 7 days to avoid API limitations,
        # ... we must calculate the remaining days at each iteration:
        remaining_days = (end_date - current_date).days
        
        # If remaining_days < 7, we download the data until end_date
        if remaining_days < 7:
            print(f"getting dates for {current_date}->{next_date}")
            data = yf.download(tickers=ticker, start=current_date, end=end_date, interval=interval)

            # We must convert the data to a specified timezone:
            data = convertTimezone('UTC', data, 'Datetime')
            
            data_list.append(data)

            # Setting current_date to end_date to exit the loop
            current_date = end_date

        # If remaining_days>=7, we download data for next 7 days:
        else:
            next_date = current_date + timedelta(days=7)

            print(f"getting dates for {current_date}->{next_date}")
            data = yf.download(tickers=ticker, start=current_date, end=next_date, interval=interval)

            # We must convert the data to a specified timezone:
            data = convertTimezone('UTC', data, 'Datetime')
            
            data_list.append(data)

            # Updating current_date for the next iteration:
            current_date = next_date

    final_data = pd.concat(data_list)
    final_data.reset_index(inplace=True)
    if 'index' in final_data.columns:  
        final_data.rename(columns={'index': 'Datetime'}, inplace=True)  
    return final_data

# Notes:
    # We must convert the data to a specified timezone, 
    # ... to ensure that all datetime values in the dataset are in the same timezone,
    # ... for consistency and comparison purposes.


Adding stock prices to a DataFrame based on matching dates

In [5]:
def addStockPrices(stockPrices,df):

    # Ensuring that both datasets have their datetime columns properly formatted as strings,
    # ... for accurate matching.
    stockPrices['Datetime'] = pd.to_datetime(stockPrices['Datetime'],utc=True)
    stockPrices['Datetime'] = stockPrices['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

    df['Date'] = pd.to_datetime(df['Date'],utc=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # This list will store retrieved stock prices:
    stock_prices = []

    for index, row in df.iterrows():

        publication_date = row['Date']
       
        # If the publication date exists in teh stockPrices DataFrame,
        # ... we retrieve the corresponding stock price from stockPrices DataFrame
        if publication_date in stockPrices['Datetime'].values:
            stock_price = stockPrices.loc[stockPrices['Datetime'] == publication_date, 'Adj Close'].values[0]
            stock_prices.append(stock_price)
        else:
            stock_prices.append(None)  
    df['stock_price'] = stock_prices
    df= df.dropna(subset=['stock_price'])
    return df

Adding stock prices after a specified time period to a DataFrame based on publication dates.

In [6]:
def addStockPricesAfter(stockPrices, df, time):
    # a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Ensuring that both datasets have their datetime columns properly formatted as strings,
    # ... for accurate matching.
    stockPrices['Datetime'] = pd.to_datetime(stockPrices['Datetime'], utc=True)
    stockPrices['Datetime'] = stockPrices['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

    df_copy['Date'] = pd.to_datetime(df_copy['Date'], utc=True)

    # Adding the specified time period to the publication dates
    df_copy['Date'] += timedelta(minutes=time)

    df_copy['Date'] = df_copy['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # This list will store retrieved stock prices:
    stock_prices = []

    for index, row in df_copy.iterrows():
        publication_date = row['Date']

        # If the publication date exists in the stockPrices DataFrame,
        # ... we retrieve the corresponding stock price from stockPrices DataFrame
        if publication_date in stockPrices['Datetime'].values:
            stock_price = stockPrices.loc[stockPrices['Datetime'] == publication_date, 'Adj Close'].values[0]
            stock_prices.append(stock_price)
        else:
            stock_prices.append(None)

    df_copy[f'stock_price_after_{time}_mins'] = stock_prices
    df_copy = df_copy.dropna(subset=[f'stock_price_after_{time}_mins'])

    # Calculating the change percentages
    change_percentages=[]

    for index, row in df_copy.iterrows():
        change_percentage=((row[f'stock_price_after_{time}_mins'] - row['stock_price'] )/ row['stock_price']) * 100
        change_percentages.append(change_percentage)

    df_copy['change_percentage']=change_percentages
    return df_copy

Labeling the Data based on stock price changes after a certain period of time.

In [7]:
def label(df,down,up):
    # List to store labels indicating stock price changes
    labels=[]

    down_threshold = down
    up_threshold = up  
    
    for index, row in df.iterrows():

        # Labeling the data:
        if row['change_percentage']>up_threshold:
            labels.append(1)
        elif row['change_percentage']<down_threshold:
            labels.append(-1)
        else:
            labels.append(0)

    df[f'label']=labels
    return df

Creating a CSV file

In [8]:
def turnToCSV(df,name):
    df.to_csv(fr'..\data\{name}.csv',index=False)

# <font color='pink'>Main Program</font>

Reading our News dataset tha contains all of the News articles and their publication dates

In [9]:
News=pd.read_csv(r'..\data\News.csv')
News.head(1)

,Date,article_title,article,source_name,source_link,ticker_symbol,Positive,Negative,Neutral,Compound,...,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220
0,2024-05-15 01:02:37+00:00,Huawei flagship store surge in China signals s...,HONG KONG (Reuters) — Huawei is revamping its ...,Yahoo Finance,https://finance.yahoo.com/news/huawei-flagship...,AAPL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creating the stockPrices dataset

In [10]:
# Getting the start date from the News dataset and the end date:
start_date = News['Date'].min().split()[0] 
end_date =  News['Date'].max().split()[0]  
print("Starting Date:",start_date)
print("Ending Date:",end_date)

Starting Date: 2024-03-05
Ending Date: 2024-05-15


In [11]:
# Creating a CSV file with all the stock prices corresponding to these dates:
#stockPrices=getStockPrices(start_date,end_date,'AAPL')

try:
    existing_data = pd.read_csv(r"..\data\StockPrices.csv")
except FileNotFoundError:
    existing_data = pd.DataFrame()

new_data=getStockPrices(start_date,end_date,'AAPL')

# Appending new data to existing DataFrame
stockPrices = pd.concat([existing_data, new_data])
stockPrices.drop_duplicates(subset='Datetime', keep='first', inplace=True)

turnToCSV(stockPrices,'stockPrices')
stockPrices

getting dates for 2024-03-05 00:00:00->2024-03-12 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m data not available for startTime=1709589600 and endTime=1710194400. The requested range must be within the last 30 days.
getting dates for 2024-03-12 00:00:00->2024-03-19 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m data not available for startTime=1710194400 and endTime=1710799200. The requested range must be within the last 30 days.
getting dates for 2024-03-19 00:00:00->2024-03-26 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m data not available for startTime=1710799200 and endTime=1711404000. The requested range must be within the last 30 days.
getting dates for 2024-03-26 00:00:00->2024-04-02 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m d

,Datetime,Open,High,Low,Close,Adj Close,Volume,Date
0,2024-03-05 14:30:00+00:00,171.000000,171.000000,170.610001,170.919998,170.919998,5958861.0,NaN
1,2024-03-05 14:31:00+00:00,171.020004,171.074997,171.020004,171.059998,171.059998,926341.0,NaN
2,2024-03-05 14:32:00+00:00,171.335999,171.339996,171.270004,171.300003,171.300003,703908.0,NaN
3,2024-03-05 14:33:00+00:00,171.929993,171.970001,171.875000,171.895004,171.895004,994237.0,NaN
4,2024-03-05 14:34:00+00:00,171.625000,171.740005,171.580002,171.610001,171.610001,626087.0,NaN
...,...,...,...,...,...,...,...,...
17108,2024-05-06 19:57:00+00:00,181.589996,181.679993,181.559998,181.669998,181.669998,325728.0,NaN
17109,2024-05-06 19:58:00+00:00,181.669998,181.695007,181.649994,181.695007,181.695007,332215.0,NaN
17110,2024-05-06 19:59:00+00:00,181.695007,181.720001,181.490005,181.699997,181.699997,1099531.0,NaN
17111,2024-05-08 20:00:00+00:00,182.740005,182.740005,182.740005,182.740005,182.740005,0.0,NaN


Getting the stock prices corresponding to the publication date and removing seconds from the date

In [12]:
# First, we need to remove the seconds from the dates since 'yfinance operates on minute-based intervals:
News=removeSeconds(News)

# Adding the stock prices corresponding to the publication date:
News=addStockPrices(stockPrices,News)
turnToCSV(News,'NewsWithStockPrices')

In [13]:
News.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403 entries, 98 to 1116
Columns: 222 entries, Date to stock_price
dtypes: float64(215), object(7)
memory usage: 702.1+ KB


Getting the stock prices after a certain amount of time from the publication date and Labeling the data.<br>
Created 3 new datasets, for clarity, each one for a specified time delay.

In [14]:
# Adding stock prices after 30 minutes of the publication date:
after_30_mins=addStockPricesAfter(stockPrices,News,30)
after_30_mins=label(after_30_mins,-0.1,0.1)
after_30_mins.to_csv(r'..\data\stockPrices\stock_prices_after_30_mins.csv',index=False)

# Adding stock prices after 6 hours (360 minutes) of the publication date:
after_6_hours=addStockPricesAfter(stockPrices,News,120)
after_6_hours=label(after_6_hours,-0.2,0.2)
after_6_hours.to_csv(r'..\data\stockPrices\stock_prices_after_6_hours.csv',index=False)

# Adding stock prices after 1 day (1440 minutes) of the publication date:
after_1_day=addStockPricesAfter(stockPrices,News,1440)
after_1_day=label(after_1_day,-0.4,0.4)
after_1_day.to_csv(r'..\data\stockPrices\stock_prices_after_1_day.csv',index=False)


In [15]:
after_30_mins.describe()

,Positive,Negative,Neutral,Compound,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,...,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,stock_price,stock_price_after_30_mins,change_percentage,label
count,380.000000,380.000000,380.000000,380.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,387.000000,387.000000,387.000000,387.000000
mean,0.105534,0.043961,0.850495,0.557863,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,171.215518,171.233943,0.011654,0.049096
std,0.069523,0.041127,0.070315,0.651293,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.870145,2.814478,0.267149,0.785337
min,0.000000,0.000000,0.424000,-0.997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,164.362396,164.964996,-1.212901,-1.000000
25%,0.066000,0.015000,0.820750,0.350450,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,169.565048,169.542496,-0.137149,-1.000000
50%,0.096000,0.036000,0.857500,0.949650,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,171.029999,171.089996,0.016624,0.000000
75%,0.126250,0.065000,0.891000,0.995225,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,172.490051,172.495255,0.160487,1.000000
max,0.576000,0.256000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,182.662506,182.315002,1.012974,1.000000


In [16]:
after_6_hours.describe()

,Positive,Negative,Neutral,Compound,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,...,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,stock_price,stock_price_after_120_mins,change_percentage,label
count,318.000000,318.000000,318.000000,318.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,319.000000,319.000000,319.000000,319.000000
mean,0.102994,0.044101,0.852918,0.573320,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,171.212515,171.145399,-0.036461,0.028213
std,0.066258,0.041977,0.068614,0.632402,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.898713,2.756956,0.534508,0.836935
min,0.000000,0.000000,0.424000,-0.997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,164.362396,164.119995,-2.334167,-1.000000
25%,0.066000,0.015250,0.822000,0.366350,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,169.425003,169.450897,-0.390668,-1.000000
50%,0.095500,0.036000,0.860000,0.951100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,170.979996,171.154999,-0.005711,0.000000
75%,0.125750,0.063000,0.891000,0.994950,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,172.442497,172.500000,0.293339,1.000000
max,0.576000,0.256000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,182.662506,181.320007,2.181137,1.000000


In [17]:
after_1_day.describe()

,Positive,Negative,Neutral,Compound,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,...,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,stock_price,stock_price_after_1440_mins,change_percentage,label
count,319.000000,319.000000,319.000000,319.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,325.000000,325.000000,325.000000,325.000000
mean,0.108539,0.042138,0.849295,0.561008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,171.202763,171.636566,0.259841,0.040000
std,0.073228,0.040268,0.073835,0.651244,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.582964,3.170930,1.564145,0.805996
min,0.000000,0.000000,0.424000,-0.997700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,165.510300,165.009995,-4.201370,-1.000000
25%,0.067500,0.011500,0.820500,0.381800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,169.389999,169.729996,-0.557123,-1.000000
50%,0.097000,0.035000,0.857000,0.946800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,170.908401,171.289993,0.114679,0.000000
75%,0.133500,0.062000,0.891000,0.995250,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,172.500000,172.785004,0.715823,1.000000
max,0.576000,0.256000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,178.379898,185.089996,7.771880,1.000000


In [18]:
after_30_mins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387 entries, 99 to 1116
Columns: 225 entries, Date to label
dtypes: float64(217), int64(1), object(7)
memory usage: 683.3+ KB


In [19]:
after_6_hours.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 100 to 1116
Columns: 225 entries, Date to label
dtypes: float64(217), int64(1), object(7)
memory usage: 563.2+ KB


In [20]:
after_1_day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 104 to 1116
Columns: 225 entries, Date to label
dtypes: float64(217), int64(1), object(7)
memory usage: 573.8+ KB
